In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from cifar10_models import mobilenetv2
from torchvision import transforms, datasets
from torch.utils.data import Subset, DataLoader, Dataset
import matplotlib.pyplot as plt

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [3]:

Net = mobilenetv2.mobilenet_v2(pretrained=True)

In [4]:
#for module in Net.modules():
#    if isinstance(module, nn.BatchNorm2d):
#        module.eval()

In [5]:
#for param in Net.parameters():
#    param.requires_grad = False


In [6]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

In [7]:
torch.cuda.memory_allocated()/1024**2, torch.cuda.memory_cached()/1024**2

(0.0, 0.0)

In [8]:
Net.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=10, bias=True)
)

In [9]:
class MyDataset(Dataset):
    def __init__(self, train_x, train_y):
        self.data = train_x 
        self.target = train_y 
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y
    
    def __len__(self):
        return len(self.data)


In [10]:
class RegressionModel(nn.Module):
    def __init__(self, pretrainedModel):
        super().__init__()
        self.features = pretrainedModel.features
        
        self.Flatten = nn.Flatten()
        self.FC1 = nn.Linear(20480, 128)
        self.FC2 = nn.Linear(128, 64)
        self.FC3 = nn.Linear(64, 32)
        self.FC4 = nn.Linear(32, 32)


        self.Dropout2D = nn.Dropout2d(p=0.2)
        self.Dropout = nn.Dropout(p=0.2)


        self.outLayer1 = nn.Linear(32, 4)
        #self.outLayer2 = nn.Linear(32, 4)
        #self.outLayer3 = nn.Linear(32, 4)
        #self.outLayer4 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.Dropout2D(self.features(x))
        x = self.Flatten(x) # flatten out the last conv layer

        x = self.Dropout(torch.selu(self.FC1(x)))# use dropout with p=0.2
        x = self.Dropout(torch.selu(self.FC2(x)))
        x = self.Dropout(torch.selu(self.FC3(x)))
        x = self.Dropout(torch.selu(self.FC4(x)))

        out1 = self.outLayer1(x)
        #out2 = self.outLayer2(x)
        #out3 = self.outLayer3(x)
        #out4 = self.outLayer4(x)

        return out1#, out2, out3, out4


In [11]:
dataset = datasets.CIFAR10('../data-unversioned/p1ch7/', train=True, transform=preprocess, download=False)
imgs = os.listdir('data/training')
imgs.sort()
indices = [int(name[0:5]) for name in imgs]
my_subset = Subset(dataset, indices) #create subset based on indices


In [12]:
train_x = []
for data, label in my_subset:
    train_x.append(data)
train_x = torch.stack(train_x)


In [13]:
labels = torch.from_numpy(np.load('data/labels.npy')).float()
labels = labels.to(device=device)

TrainSet = MyDataset(train_x, labels)

In [14]:
TrainSet[0][1]

tensor([[29.3383,  5.3111,  0.9465,  5.0000],
        [ 5.0803, 13.2412,  4.5605,  2.1632],
        [21.7605, 10.3194,  4.6590,  5.0000],
        [13.8442, 19.0300,  5.0000,  5.0000]], device='cuda:0')

In [15]:
train_loader = DataLoader(TrainSet, batch_size=64, shuffle=True) 
model = RegressionModel(Net).to(device=device)


In [16]:
img = cv2.imread('data/attacks/00000_input_img.png')
imgT = torch.tensor(img)
imgT = imgT.view(3, 32 , 32).to(device=device)
imgT = torch.unsqueeze(imgT, 0).float()
out1, out2, out3, out4 = model(imgT)

ValueError: not enough values to unpack (expected 4, got 1)

In [17]:
model.training

True

In [18]:
for batch, l in train_loader:
    A = batch.to(device=device)
    B = l
    break

A.shape
B.shape


torch.Size([64, 4, 4])

In [19]:
d = arrange_labels_correctly(B)[0]
IoULoss(A, d)

NameError: name 'arrange_labels_correctly' is not defined

In [20]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

In [21]:
def IoULoss(predictions, ground_truth):
    P_x = predictions[..., 0:1]
    P_y = predictions[..., 1:2]
    P_h = predictions[..., 2:3]
    P_w = predictions[..., 3:4]
    G_x = ground_truth[..., 0:1]
    G_y = ground_truth[..., 1:2]
    G_h = ground_truth[..., 2:3]
    G_w = ground_truth[..., 3:4]

    w_intersection = torch.min(P_x + P_w, G_x + G_w) - torch.max(P_x, G_x)
    h_intersection = torch.min(P_y + P_h, G_y + G_h) - torch.max(P_y, G_y)
    intersection = w_intersection.clamp(0) * h_intersection.clamp(0)#if no intersection, value will default to 0
   
    union = P_h * P_w + G_h * G_w - intersection
    IoU = (intersection + 1e-6)/(union + 1e-6)

    ##central points
    central_p_x = (P_x + P_w)/2
    central_p_y = (P_y + P_h)/2
    central_l_x = (G_x + G_w)/2
    central_l_y = (G_y + G_h)/2
    euc_dist = torch.sqrt(torch.square(central_l_x - central_p_x) + torch.square(central_l_y - central_p_y))
    
    
    
    #get diagional
    w_union = torch.max(P_x + P_w, G_x + G_w) - torch.min(P_x, G_x)
    h_union = torch.max(P_y + P_h, G_y + G_h) - torch.min(P_y, G_y)
    c_diag = torch.sqrt(torch.square(w_union) + torch.square(h_union))

    #penalty term
    penalty_term = euc_dist/c_diag

    DistanceIoULoss = 1 - IoU + penalty_term

    return DistanceIoULoss.mean()

In [22]:
loss = IoULoss

In [23]:
def arrange_labels_correctly(label_batch):
    l1 = torch.stack([label[0] for label in label_batch])
    l2 = torch.stack([label[1] for label in label_batch])
    l3 = torch.stack([label[2] for label in label_batch])
    l4 = torch.stack([label[3] for label in label_batch])

    return l1, l2, l3, l4

In [24]:
def training_loop(n_epochs, optimizer, model, loss_fn, loader, batch_size: int):    
    losses = []

    for epoch in range(1, n_epochs+1):
        running_loss = 0.0
        idx = 0
        for img_batch, labels in loader:
            img_batch = img_batch.to(device=device)
            #out1, out2, out3, out4 = model(img_batch)
            out1 = model(img_batch)
            label1, label2, label3, label4 = arrange_labels_correctly(labels)
            
            loss1 = loss_fn(out1, label1)
            #loss2 = loss_fn(out2, label2)
            #loss3 = loss_fn(out3, label3)
            #loss4 = loss_fn(out4, label4)

            loss_total = loss1
            #print(loss_total)
            #loss_total = (loss1 + loss2 + loss3 + loss4)#accumulate loss 
            optimizer.zero_grad()

            loss_total.backward()
            optimizer.step()
            running_loss += loss_total.item() * img_batch.size(0)
            
            if idx % 400 == 0:
                print(f'step {idx} is the current iteration and loss is: {loss_total}')

            idx += 1
            
        
        epoch_loss = running_loss / len(TrainSet)
        print(f'At epoch: {epoch}, the training loss is {epoch_loss}')
        losses.append(epoch_loss)
        

In [25]:
training_loop(n_epochs=12, optimizer=optimizer, model=model, loss_fn=loss_fn, loader=train_loader, batch_size=64)


step 0 is the current iteration and loss is: 136.04183959960938
At epoch: 1, the training loss is 95.42759398653872
step 0 is the current iteration and loss is: 69.96180725097656
At epoch: 2, the training loss is 44.0995752926168
step 0 is the current iteration and loss is: 32.76673889160156
At epoch: 3, the training loss is 38.354484032442286
step 0 is the current iteration and loss is: 34.053314208984375
At epoch: 4, the training loss is 36.44070301132736
step 0 is the current iteration and loss is: 41.09621047973633
At epoch: 5, the training loss is 35.108608035988695
step 0 is the current iteration and loss is: 43.881492614746094
At epoch: 6, the training loss is 33.77047997256925
step 0 is the current iteration and loss is: 36.84600830078125
At epoch: 7, the training loss is 32.77409259358629
step 0 is the current iteration and loss is: 32.38523864746094
At epoch: 8, the training loss is 31.51329566109653
step 0 is the current iteration and loss is: 23.24835205078125
At epoch: 9, 

In [26]:
training_loop(n_epochs=20, optimizer=optimizer, model=model, loss_fn=loss, loader=train_loader, batch_size=64)

step 0 is the current iteration and loss is: 1.3308002948760986
At epoch: 1, the training loss is 1.3488286616111271
step 0 is the current iteration and loss is: 1.3412375450134277
At epoch: 2, the training loss is 1.3508517068903756
step 0 is the current iteration and loss is: 1.3485198020935059
At epoch: 3, the training loss is 1.3460980313850808
step 0 is the current iteration and loss is: 1.3441489934921265
At epoch: 4, the training loss is 1.341671674767616
step 0 is the current iteration and loss is: 1.344853401184082
At epoch: 5, the training loss is 1.3411050069021273
step 0 is the current iteration and loss is: 1.3581230640411377
At epoch: 6, the training loss is 1.3391160496670689
step 0 is the current iteration and loss is: 1.334467887878418
At epoch: 7, the training loss is 1.3361945206365369
step 0 is the current iteration and loss is: 1.356292963027954
At epoch: 8, the training loss is 1.3368911886710546
step 0 is the current iteration and loss is: 1.3348350524902344
At e

In [ ]:
asd = my_subset[5][0].to(device=device)

In [ ]:
for imgbatch, labels in train_loader:
    o1 = model(imgbatch.to(device=device))
    
    asd = labels
    break

In [ ]:
loss(o1, l1)

In [ ]:
asd.shape

In [ ]:
l1 = torch.stack([a[0][:2] for a in asd])
l1.shape

In [ ]:

labels[0], l1[0]

In [ ]:
del model
torch.cuda.empty_cache()
del labels
torch.cuda.empty_cache()

In [27]:
model.eval()

print("hello")

hello


In [28]:
model.training

False

In [29]:
asd2 = my_subset[0][0].to(device=device)
asd3 = my_subset[84][0].to(device=device)

In [30]:
model(torch.unsqueeze(asd2, 0)), model(torch.unsqueeze(asd3, 0))

(tensor([[18.0107, 19.8563,  4.6095,  4.9596]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[11.8487, 14.1766,  3.0326,  3.6622]], device='cuda:0',
        grad_fn=<AddmmBackward>))

In [ ]:
img_p = cv2.imread('data/training/00000_input_img.png')
import matplotlib.patches as patches


In [31]:
labels[0][0], labels[84][0]

(tensor([29.3383,  5.3111,  0.9465,  5.0000], device='cuda:0'),
 tensor([ 5.4134, 20.7168,  3.7898,  0.9964], device='cuda:0'))

In [ ]:
img_tran = img_p
rect = patches.Rectangle((14.2388, 15.0060), 2.7476, 2.3851, linewidth=1, edgecolor='k', facecolor='k')
rect2 = patches.Rectangle((14.1871, 15.7513), 4.1633, 4.0735, linewidth=1, edgecolor='k', facecolor='k')
rect3 = patches.Rectangle((15.5461,  14.7842), 3.6610, 4.1208, linewidth=1, edgecolor='w', facecolor='w')
rect4 = patches.Rectangle((15.3920,  15.0777), 4.6964, 4.7229, linewidth=1, edgecolor='w', facecolor='w')

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(img_p)
ax.add_patch(rect)
ax.add_patch(rect2)
ax.add_patch(rect3)
ax.add_patch(rect4)
plt.show()